In [1]:
import os
# del os.environ['PROJ_LIB']
import pandas as pd
from datetime import date, timedelta
from pathlib import Path
import shutil 
import json
import numpy as np
import uuid
import shutil
from osgeo import gdal, gdalnumeric, ogr, osr
from pyproj import Transformer, CRS

# TOC:

 1. <a href=#Create-a-Dataframe-from-Folders>Creating a Metadata Dataframe from local source containing landset images</a>
 2. <a href=#Selecting-Images-based-on-Sample-Collection-Dates>Using Dataframes to select images based on dates</a>
     1. <a href=#Selecting-imagery-datasets-based-on-date-ranges-from-our-categorical-dataframes>Organizing sample data into categorical dataframes</a>
     1. <a href=#Identifying-if-any-of-the-images-contain-the-sample-point>Aditional filtering based using the landset image metadata</a>
 3. <a href=#Processing-Images>Clipping selected images using gdal</a>

# Default Parameters

In [2]:
# Location of Source LandSat Images
landsat_source_folder = r'\\192.168.1.5\Projects\AI-LA\spring2022\landsat8'

#............................................................................................
# DO NOT CHANGE ANYTHING BELOW. FOLDERS ARE RELATIVLY REFERENCED BASED ON PROJECT STRUCTURE.
notebook_path = os.path.dirname(os.path.realpath("__file__"))
root_path = Path(os.path.abspath("__file__")).parents[2]
shapefiles_path = os.path.join(root_path, 'data', 'shapefiles')
output_data_path = os.path.join(root_path, 'data', 'output')
#............................................................................................

def print_processing_info():
    print('........................................')
    print('      Running Image Clip Process\n')
    print(f'Project Folder Path: {root_path}')
    print(f'Shapefile Folder Path: {shapefiles_path}')
    print(f'Source Landsat Folder: {landsat_source_folder}')
    print(f'Output Folder for Clipped Images: {output_data_path}')
    print('........................................')
    
print_processing_info()

........................................
      Running Image Clip Process

Project Folder Path: C:\ai-la\landsat-processing
Shapefile Folder Path: C:\ai-la\landsat-processing\data\shapefiles
Source Landsat Folder: \\192.168.1.5\Projects\AI-LA\spring2022\landsat8
Output Folder for Clipped Images: C:\ai-la\landsat-processing\data\output
........................................


# Custom Functions

In [25]:
def organize_metadata_coordinates(dict_payload):
    pretty_dict = {'upper_left' : {'geographic': None, 'projected': None},
                   'lower_left': {'geographic': None, 'projected': None},
                   'lower_right': {'geographic': None, 'projected': None},
                   'upper_right': {'geographic': None, 'projected': None}}
    if dict_payload['geographic']:
        pretty_dict['upper_left']['geographic'] = dict_payload['geographic'][0]
        pretty_dict['lower_left']['geographic'] = dict_payload['geographic'][1]
        pretty_dict['lower_right']['geographic'] = dict_payload['geographic'][2]
        pretty_dict['upper_right']['geographic'] = dict_payload['geographic'][3]
    
    if dict_payload['projected']:
        pretty_dict['upper_left']['projected'] =  [
                                      float(dict_payload['projected']['CORNER_UL_PROJECTION_X_PRODUCT']),
                                      float(dict_payload['projected']['CORNER_UL_PROJECTION_Y_PRODUCT'])]
        pretty_dict['upper_right']['projected'] = [
                                      float(dict_payload['projected']['CORNER_UR_PROJECTION_X_PRODUCT']),
                                      float(dict_payload['projected']['CORNER_UR_PROJECTION_Y_PRODUCT'])]
        pretty_dict['lower_left']['projected'] = [
                                      float(dict_payload['projected']['CORNER_LL_PROJECTION_X_PRODUCT']),
                                      float(dict_payload['projected']['CORNER_LL_PROJECTION_Y_PRODUCT'])]
        pretty_dict['lower_right']['projected'] = [ 
                                      float(dict_payload['projected']['CORNER_LR_PROJECTION_X_PRODUCT']),
                                      float(dict_payload['projected']['CORNER_LR_PROJECTION_Y_PRODUCT'])]
    return pretty_dict
        
            

def extract_json_properties(root_folder, file_prefix):
    
    # Json files that contain the metadata coordinate info for Landsat imagery. 
    json_suffix = ['_SR_stac.json', '_MTL.json']
    
    # Dictionary that to be populated with metadata
    meta_dict = {'geographic' : None, 'projected': None}
    
    for suffix in json_suffix:
        json_file = f'{file_prefix}{suffix}'
        try:
            with open(os.path.join(root_folder, file_prefix, json_file)) as f:
                data = json.load(f)
                if 'SR'in suffix:
                    meta_dict['geographic'] = data['geometry']['coordinates'][0]
                else:
                    meta_dict['projected'] = data['LANDSAT_METADATA_FILE']['PROJECTION_ATTRIBUTES']
        except FileNotFoundError:
            print('File does not exist')
    return organize_metadata_coordinates(meta_dict)
    

def check_if_sample_within(sample_long, sample_lat, selection_df, projection_prop='geographic'):
    selected_sets = list()
    for index, row in selection_df.iterrows():
        # Set default values
        x_flag = False
        y_flag = False
        
        # Extract max/min lat long coords across the bounding coord columns
        max_long = max(
            row['upper_left'][projection_prop][0],
            row['lower_left'][projection_prop][0],
            row['lower_right'][projection_prop][0],
            row['upper_right'][projection_prop][0])
        max_lat = max(
            row['upper_left'][projection_prop][1],
            row['lower_left'][projection_prop][1],
            row['lower_right'][projection_prop][1],
            row['upper_right'][projection_prop][1])
        min_long = min(
            row['upper_left'][projection_prop][0],
            row['lower_left'][projection_prop][0],
            row['lower_right'][projection_prop][0],
            row['upper_right'][projection_prop][0])
        min_lat = min(
            row['upper_left'][projection_prop][1],
            row['lower_left'][projection_prop][1],
            row['lower_right'][projection_prop][1],
            row['upper_right'][projection_prop][1])
        
        # Check if sample coords are within the max/min ranges
        if min_long <= sample_long <= max_long:
            x_flag = True
        if min_lat <= sample_lat <= max_lat:
            y_flag = True
        if x_flag and y_flag:
            selected_sets.append(row['folder_name'])
    
    return selected_sets


def check_if_grid_within(bounding_extent, img_meta_dict, projection_prop='geographic'):
        # Set default values
        x_max_flag = False
        x_min_flag = False
        y_max_flag = False
        y_min_flag = False
        
        # Extract max/min lat long coords across the bounding coord columns
        max_long = max(
            img_meta_dict['upper_left'][projection_prop][0],
            img_meta_dict['lower_left'][projection_prop][0],
            img_meta_dict['lower_right'][projection_prop][0],
            img_meta_dict['upper_right'][projection_prop][0])
        max_lat = max(
            img_meta_dict['upper_left'][projection_prop][1],
            img_meta_dict['lower_left'][projection_prop][1],
            img_meta_dict['lower_right'][projection_prop][1],
            img_meta_dict['upper_right'][projection_prop][1])
        min_long = min(
            img_meta_dict['upper_left'][projection_prop][0],
            img_meta_dict['lower_left'][projection_prop][0],
            img_meta_dict['lower_right'][projection_prop][0],
            img_meta_dict['upper_right'][projection_prop][0])
        min_lat = min(
            img_meta_dict['upper_left'][projection_prop][1],
            img_meta_dict['lower_left'][projection_prop][1],
            img_meta_dict['lower_right'][projection_prop][1],
            img_meta_dict['upper_right'][projection_prop][1])
        
        # Check if grid coords are within the max/min ranges
        if bounding_extent[0] >= min_long:
            x_min_flag = True
        if bounding_extent[1] >= min_lat:
            y_min_flag = True
        if bounding_extent[2] <= max_long:
            x_max_flag = True
        if bounding_extent[3] <= max_lat:
            y_max_flag = True
        if x_min_flag and y_min_flag and x_max_flag and y_max_flag:
            return True
        else:
            return False


def filter_df_by_date_range(collected_sample_date, dataset_df, dataset_date_field):
    days_before = (collected_sample_date - timedelta(days=30)).isoformat()
    days_after = (collected_sample_date + timedelta(days=30)).isoformat()
    return dataset_df.loc[(dataset_df[dataset_date_field] >= days_before) & (dataset_df[dataset_date_field] < days_after)]


def create_folder(root_path, folder_name):
    folder_to_check = os.path.join(root_path, folder_name)
    if not os.path.exists(folder_to_check):
        print(f'The folder, {folder_name}, does not exist. Creating Folder...')
        os.mkdir(os.path.join(root_path, folder_name))
    return folder_to_check
        
        
def clip_image(product_input_path, product_output_path, shapefile_path=None, crop_to_Cutline=False, bbox=None):
    gdal.UseExceptions()
    if not os.path.isfile(product_output_path):
        try:
            print(f'.....Clipping {os.path.split(product_input_path)[-1]}')
            options = gdal.WarpOptions(cutlineDSName=shapefile_path, cropToCutline=crop_to_Cutline, outputBounds=bbox)
            outBand = gdal.Warp(srcDSOrSrcDSTab=product_input_path,
                                destNameOrDestDS=product_output_path,
                                options=options)
            outBand= None
        except Exception as ex:
            if ex.__doc__ != 'Unspecified run-time error.':
                template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                message = template.format(type(ex).__name__, ex.args)
                print(message)
                return ex


def shapefile_to_df(shapefile_path, fields=None, feature_bbox=False):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataSource = driver.Open(shapefile_path)
    lyr_name = os.path.basename(shapefile_path).split('.')[0]
    lyr = dataSource.GetLayer()
    
    print(f'{lyr_name} contains {lyr.GetFeatureCount()} features.\n' \
         'Converting to a Dataframe..........')
    field_names = [field.name for field in lyr.schema]
    if fields:
        if not (set(fields).issubset(field_names)):
            print(f'The following fields do not exist: {set(fields).difference(field_names)}')
            return False
        else:
            field_names = fields 
    if feature_bbox:
        field_names.append('feature_bbox')
    
    df = pd.DataFrame(columns=field_names)
    for feature in lyr:
        row = dict(feature.items())
        if feature_bbox:
            geom = feature.GetGeometryRef()
            minX, maxX, minY, maxY = geom.GetEnvelope()
            row['feature_bbox'] = (minX, minY, maxX, maxY)
        
        feature_df = pd.DataFrame.from_records([row], columns=field_names)
        df = pd.concat([feature_df, df], join='inner')
    return df


def get_pixel_value_from_coordinate(input_raster, input_coordinates=[]):
    '''
    Get pixel value(s) from a raster by passing in a list of one ore more tuples [(x,y)].  Coordinates
    must match the coordinate space of the input georeferenced raster.
    '''
    pixel_values = list()
    
    # Open raster file and load band
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(input_raster)
    band = dataset.GetRasterBand(1)
    
    # Get raster properties and read in as an array
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    data = band.ReadAsArray(0, 0, cols, rows)
    
    # Get geotransform from georeferenced coordinate space
    transform = dataset.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]
#     print(f'Pixel rows: {rows} | Pixel Columns: {cols}')
#     print(f'x-orgin upper-left corner: {xOrigin} | y-orgin upper-left corner: {yOrigin}')
#     print(f'Pixel width: {pixelWidth} | Pixel height: {pixelHeight}')
    
    # Loop through input coordainte list.
    for point in input_coordinates:
        col = int((point[0] - xOrigin) / pixelWidth)
        row = int((yOrigin - point[1] ) / pixelHeight)
#         print(f'Row: {row} - Col: {col} | Pixel Value: {data[row][col]}')
        pixel_values.append({
            'x' : point[0],
            'y' : point[1],
            'pixel_value' : data[row][col]})
    return pixel_values


def convert_coordinates(x, y, coordinate_property='projected', spatial_refernce_code=32611):
    crs = CRS.from_epsg(spatial_refernce_code)  # put your desired EPSG code here
    if coordinate_property == 'projected':
        latlon2utm = Transformer.from_crs(crs.geodetic_crs, crs)
        eastings, northings = latlon2utm.transform([y], [x])
        return (eastings[0], northings[0])
    if coordinate_property == 'geographic':
        utm2latlon = Transformer.from_crs(crs, crs.geodetic_crs)
        latitudes, longitudes = utm2latlon.transform([x], [y])
        return (longitudes[0], latitudes[0])

# Inspecting Folders

In [4]:
folders = os.listdir(landsat_source_folder)
len(folders)
print(f'Number of Folders: {len(folders)}\nFirst folder name: {folders[0]}')
[print(f'{a} | {folders[0][a]}') for a in range(len(folders[0]))]

Number of Folders: 703
First folder name: LC09_L2SP_041037_20220311_20220313_02_T1
0 | L
1 | C
2 | 0
3 | 9
4 | _
5 | L
6 | 2
7 | S
8 | P
9 | _
10 | 0
11 | 4
12 | 1
13 | 0
14 | 3
15 | 7
16 | _
17 | 2
18 | 0
19 | 2
20 | 2
21 | 0
22 | 3
23 | 1
24 | 1
25 | _
26 | 2
27 | 0
28 | 2
29 | 2
30 | 0
31 | 3
32 | 1
33 | 3
34 | _
35 | 0
36 | 2
37 | _
38 | T
39 | 1


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Folder Naming Convention:
File naming structure: __LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX__

Where:

- L = Landsat
- X = Sensor (“C”=OLI/TIRS combined, “O”=OLI-only, “T”=TIRS-only, “E”=ETM+, “T”=“TM, “M”=MSS)
- SS = Satellite (”07”=Landsat 7, “08”=Landsat 8)
- LLLL = Processing correction level (L1TP/L1GT/L1GS)
- PPP = WRS path
- RRR = WRS row
- YYYYMMDD = Acquisition year, month, day
- yyyymmdd - Processing year, month, day
- CC = Collection number (01, 02, …)
- TX = Collection category (“RT”=Real-Time, “T1”=Tier 1, “T2”=Tier 2)


Example:  __LC08_L1GT_029030_20151209_20160131_01_RT__

Means: Landsat 8; OLI/TIRS combined; processing correction level L1GT; path 029; row 030; 
acquired December 9, 2015; processed January 31, 2016; Collection 1; Real-Time



In [5]:
convention_details = {
    'satelite_type' : [0,1],
    'sensor_type' : [1,2],
    'satelite_number' : [2,4],
    'processing_correction_level' : [5, 9],
    'wrs_path' : [10,13],
    'wrs_row' : [13,16],
    'acquisition_date': [17,25],
    'processing_date': [26,34],
    'collection_number': [35,37],
    'collection_category': [38,40]
}

In [6]:
print(f'{folders[0]}\n')
for key, v in convention_details.items():
    print(f'{key} - {folders[0][v[0]:v[1]]}')

LC09_L2SP_041037_20220311_20220313_02_T1

satelite_type - L
sensor_type - C
satelite_number - 09
processing_correction_level - L2SP
wrs_path - 041
wrs_row - 037
acquisition_date - 20220311
processing_date - 20220313
collection_number - 02
collection_category - T1


In [7]:
# Converting a datetime string to a datetime column dtype
pd.to_datetime('20220313', format='%Y%m%d')

Timestamp('2022-03-13 00:00:00')

# Create a Dataframe from Folders
Converting the folder naming convention into a dataframe will make it much eaiser to perform datetime based filters so we can select the folders for image processing.

In [8]:
folder_dict = list()
for f in folders:
    details = dict()
    # Loop through and extract the components from the file name convention 
    for key, v in convention_details.items():
        details[key] = f'{f[v[0]:v[1]]}'
    
    # Adding additional details to our dictionary payload
    details['folder_name'] = f
    
    
    # Append the details payload to the list.
    folder_dict.append(details)
    # Extract corner coordinates of each each landset set.
    coords = extract_json_properties(landsat_source_folder, f)
    if coords:
        details['upper_left'] = coords['upper_left']
        details['lower_left'] = coords['lower_left']
        details['lower_right'] = coords['lower_right']
        details['upper_right'] = coords['upper_right']

dataset_pd = pd.DataFrame.from_dict(folder_dict, orient='columns')
dataset_pd

,satelite_type,sensor_type,satelite_number,processing_correction_level,wrs_path,wrs_row,acquisition_date,processing_date,collection_number,collection_category,folder_name,upper_left,lower_left,lower_right,upper_right
0,L,C,09,L2SP,041,037,20220311,20220313,02,T1,LC09_L2SP_041037_20220311_20220313_02_T1,"{'geographic': [-119.46696668761413, 34.220387...","{'geographic': [-119.92086722940347, 32.496532...","{'geographic': [-117.98781338274685, 32.119557...","{'geographic': [-117.49585332578857, 33.843505..."
1,L,C,09,L2SP,042,036,20211212,20220120,02,T1,LC09_L2SP_042036_20211212_20220120_02_T1,"{'geographic': [-120.62668974845239, 35.654627...","{'geographic': [-121.09226734219729, 33.931991...","{'geographic': [-119.12956102432886, 33.551813...","{'geographic': [-118.62358010363475, 35.274246..."
2,L,C,09,L2SP,042,036,20211228,20220121,02,T1,LC09_L2SP_042036_20211228_20220121_02_T1,"{'geographic': [-120.62237769313987, 35.654487...","{'geographic': [-121.08772162837018, 33.931872...","{'geographic': [-119.12503312800297, 33.551620...","{'geographic': [-118.6192881894316, 35.2740330..."
3,L,C,09,L2SP,042,036,20220214,20220216,02,T1,LC09_L2SP_042036_20220214_20220216_02_T1,"{'geographic': [-120.63199407138167, 35.654738...","{'geographic': [-121.09714976564294, 33.932369...","{'geographic': [-119.13182165444776, 33.551774...","{'geographic': [-118.62654228792998, 35.273935..."
4,L,C,09,L2SP,042,036,20220302,20220304,02,T1,LC09_L2SP_042036_20220302_20220304_02_T1,"{'geographic': [-120.60681975279437, 35.654684...","{'geographic': [-121.07153284197099, 33.932409...","{'geographic': [-119.10695448516327, 33.552200...","{'geographic': [-118.60148252813813, 35.274270..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,L,C,08,L2SP,040,036,20130810,20200912,02,T1,LC08_L2SP_040036_20130810_20200912_02_T1,"{'geographic': [-117.53615384822078, 35.658458...","{'geographic': [-118.00196444292065, 33.936343...","{'geographic': [-116.04490662227897, 33.554679...","{'geographic': [-115.54089807348181, 35.277018..."
699,L,C,08,L2SP,040,036,20150426,20200909,02,T1,LC08_L2SP_040036_20150426_20200909_02_T1,"{'geographic': [-117.5855356820274, 35.6582289...","{'geographic': [-118.05032112738532, 33.935670...","{'geographic': [-116.09402107581168, 33.555319...","{'geographic': [-115.59134886873088, 35.277886..."
700,L,C,08,L2SP,040,036,20160701,20200906,02,T1,LC08_L2SP_040036_20160701_20200906_02_T1,"{'geographic': [-117.51128667037634, 35.656944...","{'geographic': [-117.97792781223473, 33.934913...","{'geographic': [-116.0216591608148, 33.5528747...","{'geographic': [-115.51652983321826, 35.274828..."
701,L,C,08,L2SP,040,036,20160717,20200906,02,T1,LC08_L2SP_040036_20160717_20200906_02_T1,"{'geographic': [-117.5146008553576, 35.6569299...","{'geographic': [-117.98117034645755, 33.934617...","{'geographic': [-116.02489348339095, 33.552629...","{'geographic': [-115.51983232720802, 35.274598..."


### Update dtypes for the date columns

In [9]:
dataset_pd['acquisition_date'] = pd.to_datetime(dataset_pd['acquisition_date'], format='%Y%m%d')
dataset_pd['processing_date'] = pd.to_datetime(dataset_pd['processing_date'], format='%Y%m%d')

In [10]:
dataset_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   satelite_type                703 non-null    object        
 1   sensor_type                  703 non-null    object        
 2   satelite_number              703 non-null    object        
 3   processing_correction_level  703 non-null    object        
 4   wrs_path                     703 non-null    object        
 5   wrs_row                      703 non-null    object        
 6   acquisition_date             703 non-null    datetime64[ns]
 7   processing_date              703 non-null    datetime64[ns]
 8   collection_number            703 non-null    object        
 9   collection_category          703 non-null    object        
 10  folder_name                  703 non-null    object        
 11  upper_left                   703 non-null    

In [11]:
print(f"Let's get a breakdown of unique values per column'.\n")
# print(data.describe(include=['category']))
for field in convention_details.keys():
    unique_vals = dataset_pd[field].unique()
    print(f"\nUnique values from the {field} column ...\n")
    print(dataset_pd[field].value_counts())

Let's get a breakdown of unique values per column'.


Unique values from the satelite_type column ...

L    703
Name: satelite_type, dtype: int64

Unique values from the sensor_type column ...

C    703
Name: sensor_type, dtype: int64

Unique values from the satelite_number column ...

08    682
09     21
Name: satelite_number, dtype: int64

Unique values from the processing_correction_level column ...

L2SP    703
Name: processing_correction_level, dtype: int64

Unique values from the wrs_path column ...

040    311
041    258
042    134
Name: wrs_path, dtype: int64

Unique values from the wrs_row column ...

036    462
037    241
Name: wrs_row, dtype: int64

Unique values from the acquisition_date column ...

2022-03-11    2
2020-09-05    2
2021-09-17    2
2021-03-16    2
2021-11-04    2
             ..
2019-05-14    1
2019-05-30    1
2019-06-15    1
2019-07-01    1
2017-05-17    1
Name: acquisition_date, Length: 479, dtype: int64

Unique values from the processing_date column ...

2

# Selecting Images based on Sample Collection Dates
Now that we have our image collection metadata in a dataframe for ease of selection, let's now import our collected sample data.  What we need from our collection sample is the following:
 - Sample Date
 - XY coordinates.
 - Sample Status
 
With the above information, we create a set of dataframes based on different unique sample status values.  This give us a data structure that can be used to select images from our image metadata dataframe so they can be passed into our chipping process.

A unique ID will also need to be generated for each sample and assigned to the *sample_id* field.

In [12]:
collection_fields = ['sample_date', 'x', 'y', 'infestation_level']
nan_fields = ['sample_date', 'x', 'y']
status_field = 'infestation_level'

sample_collection_file = 'ISHB_input_file.csv'
sample_collection_df = pd.read_csv(sample_collection_file, usecols=collection_fields)

# Identifying the number of null values in the specified collection fields
print(f'{sample_collection_file} has {len(sample_collection_df)} rows')
print("The following fields contain null values:\n")
null_vals = sample_collection_df[nan_fields].isnull().sum().to_dict()
for field, count in null_vals.items():
    if count > 0:
        print(f"{field} : {count} | {sample_collection_df[field].dtype}")

# Loop through our collection fields and drop nan values:
for field in nan_fields:
    sample_collection_df.dropna(subset=field, inplace=True)

print(f'\nAfter removing null value rows, {sample_collection_file} has {len(sample_collection_df)} rows')

# Set the dtype of the sample date field to a datetime type
sample_collection_df['sample_date'] = pd.to_datetime(sample_collection_df['sample_date'])#, format='%m%d%Y')

# Generate a GUID based ID for each row in our dataset.
sample_collection_df['sample_id'] = uuid.uuid4()
sample_collection_df['sample_id'] = sample_collection_df['sample_id'].apply(lambda x: uuid.uuid4())


ISHB_input_file.csv has 410 rows
The following fields contain null values:

sample_date : 3 | object
x : 3 | float64
y : 3 | float64

After removing null value rows, ISHB_input_file.csv has 407 rows


## Split the sample collection dataframe into seperate dataframes by sample status

### Let's update the sample status field
The status field needs to be cleaned before we can create status specific dataframes.

In [13]:
print(f'Here is a list of the unique values from the field {status_field}')
print(list(sample_collection_df[status_field].unique()))

Here is a list of the unique values from the field infestation_level
['Moderate', 'Heavy', 'Low', 'Severe', nan, 'Moderate ', 'Severe. Amplifier ', 'Heavy ']


In [14]:
# Remove Trailing spaces for all values
sample_collection_df[status_field] = sample_collection_df[status_field].str.strip()

# Convert Nan to Unknown
sample_collection_df[status_field] = sample_collection_df[status_field].replace(to_replace = np.nan, value = 'Unknown')

# Convert "Severe. Amplifier" to just Severe
sample_collection_df[status_field] = sample_collection_df[status_field].apply(lambda x: str(x).replace("Severe. Amplifier", "Severe"))

print(f'Here is a list of the unique values post processing for the field {status_field}')
print(list(sample_collection_df[status_field].unique()))

Here is a list of the unique values post processing for the field infestation_level
['Moderate', 'Heavy', 'Low', 'Severe', 'Unknown']


### With cleaned up status field cleaned up, create categorical dataframes based on each health status value

In [16]:
# df.date = pd.to_datetime(df.date).values.astype(np.int64)

# pd.DataFrame(clean_sample_collection_df(df.groupby('sample_date').mean().sample_date))
x = pd.Timestamp(np.nanmean([tsp.value for tsp in clean_sample_collection_df['sample_date']]))
filtered_df = filter_df_by_date_range(x, dataset_pd, 'acquisition_date')
list(filtered_df.folder_name)

NameError: name 'clean_sample_collection_df' is not defined

In [17]:
# First let's import our cleaned up dataset
clean_sample_collection_file = 'clean_ishb_dataset.csv'
collection_fields = ['sample_date', 'x', 'y', 'infestation_level', 'sample_id']
clean_sample_collection_df = pd.read_csv(clean_sample_collection_file, usecols=collection_fields)

# Set the dtype of the sample date field to a datetime type
clean_sample_collection_df['sample_date'] = pd.to_datetime(clean_sample_collection_df['sample_date'])#, format='%m%d%Y')

# samples_status_df = {name: sample_collection_df[sample_collection_df[status_field] == name] for name in sample_collection_df[status_field].unique()}
samples_status_df = {k:v for (k, v) in clean_sample_collection_df.groupby(status_field)}

for k in samples_status_df.keys():
    print(f"\nStatus: '{k}' ({len(samples_status_df[k])} Rows)")
    print(samples_status_df[k].info())

samples_status_df[k]


Status: 'Heavy' (39 Rows)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 1 to 404
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   infestation_level  39 non-null     object        
 1   sample_date        39 non-null     datetime64[ns]
 2   x                  39 non-null     float64       
 3   y                  39 non-null     float64       
 4   sample_id          39 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 1.8+ KB
None

Status: 'Low' (115 Rows)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 2 to 406
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   infestation_level  115 non-null    object        
 1   sample_date        115 non-null    datetime64[ns]
 2   x                  115 non-null    float64       
 3  

,infestation_level,sample_date,x,y,sample_id
211,Unknown,2021-01-25,-118.055897,34.042315,96ad247b-f405-4eb8-9062-352e7f5bdc90
231,Unknown,2021-03-18,-118.067672,34.050309,c26fcdf2-6f53-47b7-a84c-c4e782c30ac8
244,Unknown,2021-03-23,-118.883573,34.040095,48514105-3c40-48dd-ad79-5771512f696f
245,Unknown,2021-03-23,-118.886423,34.046746,2466894a-3066-44be-b4e0-732aa70d3530
246,Unknown,2021-03-23,-118.886481,34.042519,080971b6-18a5-47c9-81ea-548ef4074df9
247,Unknown,2021-03-23,-118.871167,34.054437,f7e27262-8b4d-4f6c-b19f-40017a59020c
250,Unknown,2021-03-23,-118.899885,34.071823,ed1d66ff-1ef1-42cb-969a-8aee54f3a31e
254,Unknown,2021-03-23,-118.891887,34.041037,3bace547-4002-47f7-9a27-0d229bc335b9
255,Unknown,2021-03-23,-118.861619,34.036395,490ca4b7-0cb2-4145-be5a-5d807e07e5b3
256,Unknown,2021-03-23,-118.849032,34.033643,a72696ea-6aae-433c-bead-d81f291e34af


## Selecting imagery datasets based on date ranges from our categorical dataframes
The methodology for selecting imagery datasets for each sample is as follows:
 - Select datasets that fall within +-30 days of when the sample was collected
 - Create a dictionary that stores a list of image dataset names (there could be more than 1 dataset selected)
 - __STRECH GOAL__: *Mark each selected dataset if sample location falls within.  This will be useful downstream to understand how many of our samples reside within our Imagery collection dataset.*
 
The final output will be in a dataframe with the following columns:
 - Sample Date
 - Selected Image Sets: List of set names
 - Sample Within: A label that markes if the sample location is in one or more of the image sets.
 - Sample ID: GUID based ID that is uniquely tied to a sample.
 - XY Coordinates
 - Infestation Level
 

In [18]:
# Let's create an Empty dataframe to store our selection info 
empty_df_cols = ['selected_image_sets', 'sample_within']
empty_df_cols.extend(collection_fields)
empty_df = pd.DataFrame(columns = empty_df_cols)

# Create a dictionary by status that contains an empty dataframe
selected_sets_df = dict()
selected_sets_df = {key: empty_df for key in samples_status_df.keys()}

### TESTING: Loop through each sample and perform a +-30 date range filter on the image collection dataset based on the sample date

In [19]:
days_before = (clean_sample_collection_df['sample_date'][0]-timedelta(days=30)).isoformat()
days_after = (clean_sample_collection_df['sample_date'][0]+timedelta(days=30)).isoformat()
print(f"{days_before} | {clean_sample_collection_df['sample_date'][0]} | {days_after}")

2020-11-01T00:00:00 | 2020-12-01 00:00:00 | 2020-12-31T00:00:00


In [20]:
# Returning a dataframe of image products that fall within a +-30 of a specific date.
filtered_df = dataset_pd.loc[(dataset_pd['acquisition_date'] >= days_before) & (dataset_pd['acquisition_date'] < days_after)]
filtered_df

,satelite_type,sensor_type,satelite_number,processing_correction_level,wrs_path,wrs_row,acquisition_date,processing_date,collection_number,collection_category,folder_name,upper_left,lower_left,lower_right,upper_right
156,L,C,08,L2SP,040,036,2020-11-17,2021-03-15,02,T1,LC08_L2SP_040036_20201117_20210315_02_T1,"{'geographic': [-117.50896502331501, 35.656683...","{'geographic': [-117.97500056802818, 33.934395...","{'geographic': [-116.0187510097676, 33.5528516...","{'geographic': [-115.51421659642389, 35.275070..."
157,L,C,08,L2SP,040,036,2020-12-03,2021-03-13,02,T1,LC08_L2SP_040036_20201203_20210313_02_T1,"{'geographic': [-117.52918502600028, 35.657137...","{'geographic': [-117.99512964255003, 33.934776...","{'geographic': [-116.0384588780831, 33.5532769...","{'geographic': [-115.53432706110685, 35.275586..."
158,L,C,08,L2SP,040,036,2020-12-19,2021-03-10,02,T1,LC08_L2SP_040036_20201219_20210310_02_T1,"{'geographic': [-117.53150495469642, 35.657127...","{'geographic': [-117.99740472804616, 33.935028...","{'geographic': [-116.04039467439506, 33.553562...","{'geographic': [-115.53630563855158, 35.275610..."
281,L,C,08,L2SP,040,037,2020-11-17,2021-03-15,02,T1,LC08_L2SP_040037_20201117_20210315_02_T1,"{'geographic': [-117.89655384339507, 34.222047...","{'geographic': [-118.35293002196445, 32.498687...","{'geographic': [-116.42872243838295, 32.120363...","{'geographic': [-115.93567257115345, 33.843832..."
282,L,C,08,L2SP,040,037,2020-12-19,2021-03-10,02,T1,LC08_L2SP_040037_20201219_20210310_02_T1,"{'geographic': [-117.91903076607426, 34.222422...","{'geographic': [-118.37529055698865, 32.499257...","{'geographic': [-116.45066360835885, 32.120730...","{'geographic': [-115.958039911317, 33.84429365..."
412,L,C,08,L2SP,041,036,2020-11-24,2021-03-16,02,T1,LC08_L2SP_041036_20201124_20210316_02_T1,"{'geographic': [-119.06356888691536, 35.656844...","{'geographic': [-119.52893831068396, 33.934460...","{'geographic': [-117.57320226584562, 33.552978...","{'geographic': [-117.0675130808097, 35.2749792..."
413,L,C,08,L2SP,041,036,2020-12-26,2021-03-10,02,T1,LC08_L2SP_041036_20201226_20210310_02_T1,"{'geographic': [-119.07285687120702, 35.657225...","{'geographic': [-119.53836682479624, 33.935077...","{'geographic': [-117.58160803115088, 33.553480...","{'geographic': [-117.0760899077369, 35.2752446..."
501,L,C,08,L2SP,041,037,2020-11-24,2021-03-16,02,T1,LC08_L2SP_041037_20201124_20210316_02_T1,"{'geographic': [-119.45252941793292, 34.222165...","{'geographic': [-119.90705035139283, 32.498610...","{'geographic': [-117.98193516040185, 32.120009...","{'geographic': [-117.48888110839788, 33.843398..."
502,L,C,08,L2SP,041,037,2020-12-26,2021-03-10,02,T1,LC08_L2SP_041037_20201226_20210310_02_T1,"{'geographic': [-119.46165658795262, 34.222523...","{'geographic': [-119.91599803609586, 32.498944...","{'geographic': [-117.99020843174912, 32.120486...","{'geographic': [-117.49763907041219, 33.843904..."
617,L,C,08,L2SP,042,036,2020-11-15,2021-03-15,02,T1,LC08_L2SP_042036_20201115_20210315_02_T1,"{'geographic': [-120.5958178223164, 35.6567713...","{'geographic': [-121.06141710766308, 33.934501...","{'geographic': [-119.10519464241375, 33.552906...","{'geographic': [-118.59935253215986, 35.274974..."


In [21]:
# Checking each image product to identify if the sample falls within.
selection_list = check_if_sample_within(clean_sample_collection_df.loc[0]['x'], clean_sample_collection_df.loc[0]['y'], filtered_df)
selection_list

['LC08_L2SP_040037_20201117_20210315_02_T1',
 'LC08_L2SP_040037_20201219_20210310_02_T1',
 'LC08_L2SP_041036_20201124_20210316_02_T1',
 'LC08_L2SP_041036_20201226_20210310_02_T1',
 'LC08_L2SP_041037_20201124_20210316_02_T1',
 'LC08_L2SP_041037_20201226_20210310_02_T1',
 'LC08_L2SP_040037_20201203_20210313_02_T1']

## Identifying if any of the images contain the sample point
 1. Get X and Y coordinantes from sample
 2. Check if x coord is within the max/min longitude range
 3. check if y coord is within the max/min latitude range
 4. If both checks pass, mark "sample_within" column as yes, otherwise no.

*Note: If either check does not pass, we can break out and automatically mark as no*


In [22]:
for key in samples_status_df.keys():
    print(f'Working through the {key} category...')
    
    for row in samples_status_df[key].itertuples():
        # save namedtuple to a dictionary object. This dictionary will be updated with additional keys
        row_dict = row._asdict()
#         row_dict.pop('Index', None)
        
        # Return filtered dataframe from the collected sample date
        filtered_df = filter_df_by_date_range(row.sample_date, dataset_pd, 'acquisition_date')
        
        row_dict['selected_image_sets'] = check_if_sample_within(row.x, row.y, filtered_df)
        if  row_dict['selected_image_sets']:
            row_dict['sample_within'] = True
        else:
            row_dict['sample_within'] = False
        row_df = pd.DataFrame.from_records([row_dict], columns=row_dict.keys())
        selected_sets_df[key] = pd.concat([row_df, selected_sets_df[key]], join='inner')

Working through the Heavy category...
Working through the Low category...
Working through the Moderate category...
Working through the Severe category...
Working through the Unknown category...


In [23]:
# Quick test checking out the query method for a dataframe
selected_sets_df['Severe'].query('sample_id == "9e3c1c9d-4eca-488c-8893-aba1a977c7e5"')

,infestation_level,sample_date,x,y,sample_id,selected_image_sets,sample_within
0,Severe,2020-12-17 00:00:00,-117.806749,34.111714,9e3c1c9d-4eca-488c-8893-aba1a977c7e5,"[LC08_L2SP_040036_20201117_20210315_02_T1, LC0...",True


# Processing Images

Now that we have identified the images that need to be processed for chipping, we will next walk through a quick test to understand the methadology approach for chipping images.

 1. <a href=#Clipping-Test-Grids>Clipping Images for Prediction Grids</a>
 1. <a href=#Clipping-Prediction-Grids>Clipping Images for Prediction Grids</a>

### Clipping Test Grids

In [28]:
# Path to shapefiles
grid_size = '1000m'
training_data_grid_folder = os.path.join(shapefiles_path, 'training_grids')
training_shapefile = os.path.join(training_data_grid_folder, f'Training_Grid_{grid_size}.shp')
training_data_output_folder = os.path.join(output_data_path, 'chipped_training_data', grid_size)

# Convert Shapefile to dataframe
shapefile_df = shapefile_to_df(training_shapefile, feature_bbox=True)

# Image suffix names to clip
image_products = [
    '_SR_B1.TIF',
    '_SR_B2.TIF',
    '_SR_B3.TIF',
    '_SR_B4.TIF',
    '_SR_B5.TIF',
    '_SR_B6.TIF',
    '_SR_B7.TIF'
]

        
def print_processing_info():
    print('........................................')
    print('      Running Image Clip Process\n')
    print(f'Shapefile Folder Path: {training_data_grid_folder}')
    print(f'Source Landsat Folder: {landsat_source_folder}')
    print(f'Output Folder for Clipped Images: {training_data_output_folder}')
    print('........................................')

print_processing_info()
# Looping through sample status df that's grouped by category
for key, df in selected_sets_df.items():
    print(f'\nWorking through the {key} category...')
    output_category_folder_path = create_folder(training_data_output_folder, key)
    
    for row in df.itertuples():
        print(f'Clipping {len(row.selected_image_sets)} image sets for {row.sample_id}')
        for image_set in row.selected_image_sets:
            feature_extent = list(shapefile_df.query(f'sample_id == "{str(row.sample_id)}"')['feature_bbox'].values)[0]
            within_check = check_if_grid_within(feature_extent, dataset_pd.query(f'folder_name == "{image_set}"').to_dict('records')[0], projection_prop='projected')
            if within_check:
                # Construct path to image dataset
                imageset_path = os.path.join(landsat_source_folder, image_set)

                # Create chipped imageset folder if it does not exist.
                output_chipped_folder_path = create_folder(os.path.join(
                    training_data_output_folder, key),
                    f'{image_set}~{str(row.sample_id)}')

                for img in image_products:
                    img_file_path = os.path.join(imageset_path, f'{image_set}{img}')
                    
                    # While grid may reside witin the bounds of the image, if it's within a null vall fill zone, we do not want
                    # to keep the imageset as it contains bad data i.e. null picture.
                    easting, northing = convert_coordinates(row.x, row.y, coordinate_property='projected')
                    pixel_check_list = get_pixel_value_from_coordinate(img_file_path, input_coordinates=[(easting, northing)])
                    if pixel_check_list[0]['pixel_value'] <= 1:
                        print(f'{str(row.sample_id)} contains null values in {image_set}. Deleting folder\n')
                        shutil.rmtree(output_chipped_folder_path)
                        break
                    
                    output_chipped_image_path = os.path.join(output_chipped_folder_path, f'{image_set}{img}')
                    tmp = clip_image(img_file_path, output_chipped_image_path, crop_to_Cutline=True, bbox=feature_extent)


Training_Grid_1000m contains 407 features.
Converting to a Dataframe..........
........................................
      Running Image Clip Process

Shapefile Folder Path: C:\ai-la\landsat-processing\data\shapefiles\training_grids
Source Landsat Folder: \\192.168.1.5\Projects\AI-LA\spring2022\landsat8
Output Folder for Clipped Images: C:\ai-la\landsat-processing\data\output\chipped_training_data\1000m
........................................

Working through the Heavy category...
Clipping 6 image sets for 5cfafb12-5ce4-4954-8922-c781c16f52b9
The folder, LC08_L2SP_041037_20201124_20210316_02_T1~5cfafb12-5ce4-4954-8922-c781c16f52b9, does not exist. Creating Folder...
5cfafb12-5ce4-4954-8922-c781c16f52b9 contains null values in LC08_L2SP_041037_20201124_20210316_02_T1. Deleting folder

The folder, LC08_L2SP_041037_20201226_20210310_02_T1~5cfafb12-5ce4-4954-8922-c781c16f52b9, does not exist. Creating Folder...
5cfafb12-5ce4-4954-8922-c781c16f52b9 contains null values in LC08_L2SP_0410

.....Clipping LC08_L2SP_041036_20210924_20210930_02_T1_SR_B7.TIF
The folder, LC08_L2SP_041036_20211010_20211019_02_T1~ba08fcbd-affc-401c-8eb9-14769f193664, does not exist. Creating Folder...
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B1.TIF
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B2.TIF
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B3.TIF
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B4.TIF
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B5.TIF
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B6.TIF
.....Clipping LC08_L2SP_041036_20211010_20211019_02_T1_SR_B7.TIF
The folder, LC08_L2SP_041037_20211010_20211019_02_T1~ba08fcbd-affc-401c-8eb9-14769f193664, does not exist. Creating Folder...
ba08fcbd-affc-401c-8eb9-14769f193664 contains null values in LC08_L2SP_041037_20211010_20211019_02_T1. Deleting folder

The folder, LC08_L2SP_040037_20210917_20210925_02_T1~ba08fcbd-affc-401c-8eb9-14769f193664, does not exist. C

.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B1.TIF
.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B2.TIF
.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B3.TIF
.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B4.TIF
.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B5.TIF
.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B6.TIF
.....Clipping LC08_L2SP_041036_20210620_20210629_02_T1_SR_B7.TIF
The folder, LC08_L2SP_041036_20210706_20210713_02_T1~d8055ece-5353-43a8-8be4-663ae16301e5, does not exist. Creating Folder...
.....Clipping LC08_L2SP_041036_20210706_20210713_02_T1_SR_B1.TIF
.....Clipping LC08_L2SP_041036_20210706_20210713_02_T1_SR_B2.TIF
.....Clipping LC08_L2SP_041036_20210706_20210713_02_T1_SR_B3.TIF
.....Clipping LC08_L2SP_041036_20210706_20210713_02_T1_SR_B4.TIF
.....Clipping LC08_L2SP_041036_20210706_20210713_02_T1_SR_B5.TIF
.....Clipping LC08_L2SP_041036_20210706_20210713_02_T1_SR_B6.TIF
.....Clipping LC08_L2SP_04103

.....Clipping LC08_L2SP_041036_20210807_20210818_02_T1_SR_B5.TIF
.....Clipping LC08_L2SP_041036_20210807_20210818_02_T1_SR_B6.TIF
.....Clipping LC08_L2SP_041036_20210807_20210818_02_T1_SR_B7.TIF
Clipping 8 image sets for b30fe4b2-4f94-4ddb-90a5-af505117c429
The folder, LC08_L2SP_040037_20210410_20210416_02_T1~b30fe4b2-4f94-4ddb-90a5-af505117c429, does not exist. Creating Folder...
b30fe4b2-4f94-4ddb-90a5-af505117c429 contains null values in LC08_L2SP_040037_20210410_20210416_02_T1. Deleting folder

The folder, LC08_L2SP_040037_20210512_20210524_02_T1~b30fe4b2-4f94-4ddb-90a5-af505117c429, does not exist. Creating Folder...
b30fe4b2-4f94-4ddb-90a5-af505117c429 contains null values in LC08_L2SP_040037_20210512_20210524_02_T1. Deleting folder

The folder, LC08_L2SP_041036_20210417_20210424_02_T1~b30fe4b2-4f94-4ddb-90a5-af505117c429, does not exist. Creating Folder...
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B1.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B

.....Clipping LC08_L2SP_041036_20210316_20210328_02_T1_SR_B7.TIF
The folder, LC08_L2SP_041036_20210417_20210424_02_T1~9207d6ab-20b6-4f30-8f23-c4bc0eb8a215, does not exist. Creating Folder...
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B1.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B2.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B3.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B4.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B5.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B6.TIF
.....Clipping LC08_L2SP_041036_20210417_20210424_02_T1_SR_B7.TIF
The folder, LC08_L2SP_041037_20210316_20210328_02_T1~9207d6ab-20b6-4f30-8f23-c4bc0eb8a215, does not exist. Creating Folder...
9207d6ab-20b6-4f30-8f23-c4bc0eb8a215 contains null values in LC08_L2SP_041037_20210316_20210328_02_T1. Deleting folder

The folder, LC08_L2SP_041037_20210401_20210409_02_T1~9207d6ab-20b6-4f30-8f23-c4bc0eb8a215, does not exist. C

...

Clipping 11 image sets for 814824d8-3366-4acc-a2c6-3483bdb14393
814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_040037_20210221_20210303_02_T1. Deleting folder

814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_040037_20210309_20210317_02_T1. Deleting folder

814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_041037_20210212_20210301_02_T1. Deleting folder

814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_041037_20210228_20210311_02_T1. Deleting folder

814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_041037_20210316_20210328_02_T1. Deleting folder

814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_041037_20210401_20210409_02_T1. Deleting folder

814824d8-3366-4acc-a2c6-3483bdb14393 contains null values in LC08_L2SP_040037_20210205_20210302_02_T1. Deleting folder

Clipping 9 image sets for 08d8fcb3-c8b0-4f41-bc43-972af5547e7b
08d8fcb3-c8b0-4f41-bc43-972af5547

18315025-aa59-423e-98b5-5edfd60e65ad contains null values in LC08_L2SP_041037_20210111_20210308_02_T1. Deleting folder

18315025-aa59-423e-98b5-5edfd60e65ad contains null values in LC08_L2SP_041037_20210212_20210301_02_T1. Deleting folder

18315025-aa59-423e-98b5-5edfd60e65ad contains null values in LC08_L2SP_040037_20210205_20210302_02_T1. Deleting folder

Clipping 8 image sets for 660eccf2-b80e-489b-96ce-d0b92d7d41ea
660eccf2-b80e-489b-96ce-d0b92d7d41ea contains null values in LC08_L2SP_040037_20210104_20210309_02_T1. Deleting folder

660eccf2-b80e-489b-96ce-d0b92d7d41ea contains null values in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting folder

660eccf2-b80e-489b-96ce-d0b92d7d41ea contains null values in LC08_L2SP_041037_20210111_20210308_02_T1. Deleting folder

660eccf2-b80e-489b-96ce-d0b92d7d41ea contains null values in LC08_L2SP_041037_20210212_20210301_02_T1. Deleting folder

660eccf2-b80e-489b-96ce-d0b92d7d41ea contains null values in LC08_L2SP_040037_20210205_20210302_0

Clipping 6 image sets for 8146d8a7-6004-4000-bb5a-c7a149d1726f
8146d8a7-6004-4000-bb5a-c7a149d1726f contains null values in LC08_L2SP_040037_20201219_20210310_02_T1. Deleting folder

8146d8a7-6004-4000-bb5a-c7a149d1726f contains null values in LC08_L2SP_040037_20210104_20210309_02_T1. Deleting folder

8146d8a7-6004-4000-bb5a-c7a149d1726f contains null values in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting folder

8146d8a7-6004-4000-bb5a-c7a149d1726f contains null values in LC08_L2SP_041037_20210111_20210308_02_T1. Deleting folder

Clipping 6 image sets for 46a50abd-885a-4891-8556-db8083dddf12
46a50abd-885a-4891-8556-db8083dddf12 contains null values in LC08_L2SP_040037_20201219_20210310_02_T1. Deleting folder

46a50abd-885a-4891-8556-db8083dddf12 contains null values in LC08_L2SP_040037_20210104_20210309_02_T1. Deleting folder

46a50abd-885a-4891-8556-db8083dddf12 contains null values in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting folder

46a50abd-885a-4891-8556-db8083dddf

7f973926-4b90-48c0-adf5-c38020d07e20 contains null values in LC08_L2SP_041037_20201124_20210316_02_T1. Deleting folder

7f973926-4b90-48c0-adf5-c38020d07e20 contains null values in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting folder

7f973926-4b90-48c0-adf5-c38020d07e20 contains null values in LC08_L2SP_041037_20210111_20210308_02_T1. Deleting folder

7f973926-4b90-48c0-adf5-c38020d07e20 contains null values in LC08_L2SP_040037_20201203_20210313_02_T1. Deleting folder

Clipping 10 image sets for 6cdeb779-9f3f-4d3a-a22c-d32f97cd7e3e
6cdeb779-9f3f-4d3a-a22c-d32f97cd7e3e contains null values in LC08_L2SP_040037_20201117_20210315_02_T1. Deleting folder

6cdeb779-9f3f-4d3a-a22c-d32f97cd7e3e contains null values in LC08_L2SP_040037_20201219_20210310_02_T1. Deleting folder

6cdeb779-9f3f-4d3a-a22c-d32f97cd7e3e contains null values in LC08_L2SP_040037_20210104_20210309_02_T1. Deleting folder

6cdeb779-9f3f-4d3a-a22c-d32f97cd7e3e contains null values in LC08_L2SP_041037_20201124_20210316_

a595db5e-acf1-4925-81e9-19421f121a78 contains null values in LC08_L2SP_041037_20210401_20210409_02_T1. Deleting folder

a595db5e-acf1-4925-81e9-19421f121a78 contains null values in LC08_L2SP_041037_20210503_20210508_02_T1. Deleting folder

Clipping 9 image sets for bc94d6d9-b00f-4200-9dbc-fe67d3ebae27
bc94d6d9-b00f-4200-9dbc-fe67d3ebae27 contains null values in LC08_L2SP_042036_20210323_20210402_02_T1. Deleting folder

bc94d6d9-b00f-4200-9dbc-fe67d3ebae27 contains null values in LC08_L2SP_042036_20210408_20210416_02_T1. Deleting folder

Clipping 9 image sets for a72696ea-6aae-433c-bead-d81f291e34af
a72696ea-6aae-433c-bead-d81f291e34af contains null values in LC08_L2SP_042036_20210323_20210402_02_T1. Deleting folder

a72696ea-6aae-433c-bead-d81f291e34af contains null values in LC08_L2SP_042036_20210408_20210416_02_T1. Deleting folder

Clipping 9 image sets for 490ca4b7-0cb2-4145-be5a-5d807e07e5b3
490ca4b7-0cb2-4145-be5a-5d807e07e5b3 contains null values in LC08_L2SP_042036_20210323_2021

In [700]:
img_prod = 'LC08_L2SP_041036_20201226_20210310_02_T1'
file_name = f'{img_prod}_QA_PIXEL.TIF'
filename = os.path.join(landsat_source_folder, img_prod, file_name)

easting = 350983.8149234585
northing = 3779304.607165886
lat, long = convert_coordinates(easting, northing, coordinate_property='geographic')
print(lat, long)
pixel_check_list = get_pixel_value_from_coordinate(filename, input_coordinates=[(easting, northing)])
print(pixel_check_list)
if 1 in [x['pixel_value'] for x in pixel_check_list]:
    print(f'LC08_L2SP_041037_20210111_20210308_02_T1 within null value fill in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting image folder set.')
#     shutil.rm

-118.6162933 34.14401899999999
[{'x': 350983.8149234585, 'y': 3779304.607165886, 'pixel_value': 54534}]


In [708]:
min([x['pixel_value'] for x in pixel_check_list])

54534

In [707]:
pixel_check_list[0]['pixel_value']

54534

### Clipping Prediction Grids

In [688]:
# Path to shapefiles
grid_size = '1000m'
prediction_data_grid_folder = os.path.join(shapefiles_path, 'prediction_grids')
prediction_shapefile = os.path.join(prediction_data_grid_folder, f'Prediction_grid_{grid_size}.shp')
prediction_data_output_folder = os.path.join(output_data_path, 'chipped_prediction_data', grid_size)

#..............................................................
# COMMENT THIS AREA OUT IF YOU WANT TO TEST WITH ONE IMAGE SET
#..............................................................
#
image_set = 'LC08_L2SP_041036_20210316_20210328_02_T1'
imageset_path = os.path.join(landsat_source_folder, image_set)
#
#..............................................................

# Convert Shapefile to dataframe
shapefile_df = shapefile_to_df(prediction_shapefile, feature_bbox=True)

# Image suffix names to clip
image_products = [
    '_SR_B1.TIF',
    '_SR_B2.TIF',
    '_SR_B3.TIF',
    '_SR_B4.TIF',
    '_SR_B5.TIF',
    '_SR_B6.TIF',
    '_SR_B7.TIF'
]

        
def print_processing_info():
    print('........................................')
    print('      Running Image Clip Process\n')
    print(f'Shapefile Folder Path: {shapefile_folder}')
    print(f'Source Landsat Folder: {imageset_path}')
    print(f'Output Folder for Clipped Images: {prediction_data_output_folder}')
    print('........................................\n')

print_processing_info()
for row in shapefile_df.itertuples():
    feature_extent = list(shapefile_df.query(f'Grid == "{str(row.Grid)}"')['feature_bbox'].values)[0]
    within_check = check_if_grid_within(feature_extent, dataset_pd.query(f'folder_name == "{image_set}"').to_dict('records')[0], projection_prop='projected')
    if within_check:
        # Create chipped imageset folder if it does not exist.
        output_chipped_folder_path = create_folder(prediction_data_output_folder, f'{image_set}~{str(row.Grid)}')
        for img in image_products:
            img_file_path = os.path.join(imageset_path, f'{image_set}{img}')
            
            # While grid may reside witin the bounds of the image, if it's within a null vall fill zone, we do not want
            # to keep the imageset as it contains bad data i.e. null picture.
            coords_check_list = [(feature_extent[0], feature_extent[1]), (feature_extent[2], feature_extent[3])]                        
            pixel_check_list = get_pixel_value_from_coordinate(img_file_path, input_coordinates=coords_check_list)
            if 1 <= min([x['pixel_value'] for x in pixel_check_list]):
                print(f'{row.Grid} within null value fill in {image_set}. Deleting image folder set.')
                shutil.rmtree(output_chipped_folder_path)
                break
            
            output_chipped_image_path = os.path.join(output_chipped_folder_path, f'{image_set}{img}')
            clip_image(img_file_path, output_chipped_image_path, bbox=feature_extent)
                                                       

Prediction_grid_1000m contains 293 features.
Converting to a Dataframe..........
........................................
      Running Image Clip Process

Shapefile Folder Path: C:\ai-la\landsat-processing\data\shapefiles\training_grids\Prediction_grid_1000m.shp
Source Landsat Folder: \\192.168.1.5\Projects\AI-LA\spring2022\landsat8\LC08_L2SP_041036_20210316_20210328_02_T1
Output Folder for Clipped Images: C:\ai-la\landsat-processing\data\output\chipped_prediction_data\1000m
........................................

The folder, LC08_L2SP_041036_20210316_20210328_02_T1~BM107, does not exist. Creating Folder...
BM107 within null value fill in LC08_L2SP_041036_20210316_20210328_02_T1. Deleting image folder set.
C:\ai-la\landsat-processing\data\output\chipped_prediction_data\1000m\LC08_L2SP_041036_20210316_20210328_02_T1~BM107
The folder, LC08_L2SP_041036_20210316_20210328_02_T1~BM106, does not exist. Creating Folder...
BM106 within null value fill in LC08_L2SP_041036_20210316_20210328_02

KeyboardInterrupt: 

In [706]:
pixel_check_list[0]['pixel_value']

54534

In [678]:
easting, northing = convert_coordinates(-118.6162933, 34.144019, coordinate_property='projected')
print(easting, northing)

350983.8149234585 3779304.607165886


In [680]:
file_name = 'LC08_L2SP_041037_20210111_20210308_02_T1_QA_PIXEL.TIF'
filename = os.path.join(landsat_source_folder, 'LC08_L2SP_041037_20210111_20210308_02_T1', file_name)
pixel_check_list = get_pixel_value_from_coordinate(filename, input_coordinates=[(easting, northing)])
print(pixel_check_list)
if 1 in [x['pixel_value'] for x in pixel_check_list]:
    print(f'LC08_L2SP_041037_20210111_20210308_02_T1 within null value fill in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting image folder set.')
#     shutil.rmtree(r'C:\ai-la\landsat-processing\data\output\chipped_training_data\1000m\Heavy\LC08_L2SP_041037_20201226_20210310_02_T1~5cfafb12-5ce4-4954-8922-c781c16f52b9')

[{'x': 350983.8149234585, 'y': 3779304.607165886, 'pixel_value': 1}]
LC08_L2SP_041037_20210111_20210308_02_T1 within null value fill in LC08_L2SP_041037_20201226_20210310_02_T1. Deleting image folder set.


In [ ]:
img_file_path = os.path.join(imageset_path, f'{image_set}{img}')
                    
# While grid may reside witin the bounds of the image, if it's within a null vall fill zone, we do not want
# to keep the imageset as it contains bad data i.e. null picture.
easting, northing = convert_coordinates(row.x, row.y, coordinate_property='projected')
pixel_check_list = get_pixel_value_from_coordinate(img_file_path, input_coordinates=[(easting, northing)])
if 0 in [x['pixel_value'] for x in pixel_check_list]:
    print(f'{str(row.sample_id)} within null value fill in {image_set}. Deleting image folder set.')
    shutil.rmtree(output_chipped_folder_path)
    break
                    

In [664]:
output_chipped_folder_path

'C:\\ai-la\\landsat-processing\\data\\output\\chipped_training_data\\1000m\\Heavy\\LC08_L2SP_041037_20210401_20210409_02_T1~5df1f47a-9744-4152-849e-e1f91ef02a10'

In [663]:
pixel_check_list

[{'x': 401888.8059278651, 'y': 3767164.537177198, 'pixel_value': 0}]

In [662]:
(easting, northing)

(401888.8059278651, 3767164.537177198)

In [579]:
pixel_vals = [x['pixel_value'] for x in pixel_check_list]
pixel_vals
if 0 in pixel_vals:
    print(True)

True


# Misc.

In [145]:
driver = ogr.GetDriverByName ("ESRI Shapefile")
# ogr_ds = driver.Open(filepath)

# sf_path = r"C:\ai-la\landsat-processing\data\shapefiles\tmp\Test_100k_cells.shp"
sf = r"C:\ai-la\landsat-processing\data\shapefiles\training_grids\5cfafb12-5ce4-4954-8922-c781c16f52b9.shp"
lyr_name = os.path.basename(sf_path).split('.')[0]
sf = driver.Open(sf_path)
sql = "SELECT * FROM {} WHERE Grid LIKE '%{}%'".format(lyr_name, 'AWP')
layer = sf.ExecuteSQL(sql)

layer.GetFeatureCount()
tmp = sf.GetLayer()
tmp.SetAttributeFilter

NameError: name 'sf_path' is not defined

In [547]:
# from pyproj import Transformer, CRS

# def get_pixel_value_from_coordinate(input_raster, input_coordinates=[]):
    
#     # Open raster file and load band
#     driver = gdal.GetDriverByName('GTiff')
#     dataset = gdal.Open(filename)
#     band = dataset.GetRasterBand(1)
    
#     # Get raster properties and read in as an array
#     cols = dataset.RasterXSize
#     rows = dataset.RasterYSize
#     data = band.ReadAsArray(0, 0, cols, rows)
    
#     # Get geotransform from georeferenced coordinate space
#     transform = dataset.GetGeoTransform()
#     xOrigin = transform[0]
#     yOrigin = transform[3]
#     pixelWidth = transform[1]
#     pixelHeight = -transform[5]
#     print(f'Pixel rows: {rows} | Pixel Columns: {cols}')
#     print(f'x-orgin upper-left corner: {xOrigin} | y-orgin upper-left corner: {yOrigin}')
#     print(f'Pixel width: {pixelWidth} | Pixel height: {pixelHeight}')
    
#     # Loop through input coordainte list.
#     for point in input_coordinates:
#         col = int((point[0] - xOrigin) / pixelWidth)
#         row = int((yOrigin - point[1] ) / pixelHeight)
#         print(f'Row: {row} - Col: {col} | Pixel Value: {data[row][col]}')


# def convert_coordinates(x, y, coordinate_property='projected', spatial_refernce_code=32611):
#     crs = CRS.from_epsg(spatial_refernce_code)  # put your desired EPSG code here
#     if coordinate_property == 'projected':
#         latlon2utm = Transformer.from_crs(crs.geodetic_crs, crs)
#         eastings, northings = latlon2utm.transform([y], [x])
#         return (eastings[0], northings[0])
#     if coordinate_property == 'geographic':
#         utm2latlon = Transformer.from_crs(crs, crs.geodetic_crs)
#         latitudes, longitudes = utm2latlon.transform([x], [y])
#         return (longitudes[0], latitudes[0])

In [603]:
y, x = (34.04466382864824, -118.06315889999769)
easting, northing = convert_coordinates(x, y, coordinate_property='projected')
print(f'{easting}, {northing}')
# a = get_pixel_value_from_coordinate(filename, input_coordinates=[(easting, northing), (easting, northing)])

# lat, long = convert_coordinates(easting, northing, coordinate_property='geographic')
# print(f'{long}, {lat}')

# file_name = 'LC08_L2SP_041037_20201226_20210310_02_T1_SR_B1.TIF'
# filename = os.path.join(landsat_source_folder, 'LC08_L2SP_041037_20201226_20210310_02_T1', file_name)  #path to raster
# a = get_pixel_value_from_coordinate(filename, input_coordinates=[(easting, northing), (easting, northing)])
# print(a)

file_name = 'LC08_L2SP_041036_20210316_20210328_02_T1_SR_B1.TIF'
filename = os.path.join(landsat_source_folder, 'LC08_L2SP_041036_20210316_20210328_02_T1', file_name)  #path to raster
b = get_pixel_value_from_coordinate(filename, input_coordinates=[(easting, northing)])
print(b)

# file_name = 'LC08_L2SP_040036_20130420_20200913_02_T1_SR_B1.TIF'
# filename = os.path.join(landsat_source_folder, 'LC08_L2SP_040036_20130420_20200913_02_T1', file_name)  #path to raster
# c = get_pixel_value_from_coordinate(filename, input_coordinates=[(easting, northing)])
# print(c)

401868.8763764264, 3767617.968375425
Pixel rows: 7891 | Pixel Columns: 7751
x-orgin upper-left corner: 264585.0 | y-orgin upper-left corner: 3948615.0
Pixel width: 30.0 | Pixel height: 30.0
Row: 6033 - Col: 4576 | Pixel Value: 8664
[{'x': 401868.8763764264, 'y': 3767617.968375425, 'pixel_value': 8664}]


In [489]:
points_list = [ (-118.0631589, 34.04466498)]
x = utmToLatLng(11, eastings[0], northings[0], northernHemisphere=True)
x

(34.04466382864824, -118.06315889999769)

In [491]:
utm2latlon

<Conversion Transformer: pipeline>
Description: Inverse of UTM zone 11N
Area of Use:
- name: Between 120°W and 114°W, northern hemisphere between equator and 84°N, onshore and offshore.
- bounds: (-120.0, 0.0, -114.0, 84.0)